In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
import torch
import os
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import torch.nn.functional as l1_loss
from tqdm import tqdm

In [18]:
device = torch.device('cuda:1')
inputdir = '../data/constructed/'
resultdir = '../data/result/'
num_nodes = 4634
batchsize = 1024
# [53, 1730], [7006,1500]
total_num_timesteps = 7006
num_timesteps =  2

feature_names = [
    'iu_ac',
    'hour_sin', 'hour_cos', 
    'day_of_week_sin', 'day_of_week_cos', 
    'month_sin', 'month_cos',
    'etat_barre_0', 'etat_barre_1', 'etat_barre_2', 'etat_barre_3',
    'constructed'
]

if torch.cuda.is_available():
    print("Available CUDA devices:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")
if not os.path.exists(resultdir):
    os.makedirs(resultdir)

Available CUDA devices: 2
Device 0: NVIDIA GeForce RTX 4090
Device 1: NVIDIA GeForce RTX 4090


In [19]:
def load_dataset(filepath, feature_names, target_name='target', batch_size=batchsize, shuffle=False, train=True):
    # features = load_time_series_data(filepath, feature_names)
    data = pd.read_csv(filepath)
    features = data[feature_names].values
    # 假设每个样本都是按时间步骤连续的，此处可能需要根据实际情况调整
    features = features.reshape(-1, num_timesteps, num_nodes,  len(feature_names))
    features_tensor = torch.tensor(features, dtype=torch.float32)
    print(features_tensor.shape)

    if train:
        targets = data[target_name].values
        targets = targets.reshape(-1, num_timesteps, num_nodes, 1)
        targets_tensor = torch.tensor(targets, dtype=torch.float32)
        dataset = TensorDataset(features_tensor, targets_tensor)
    else:
        dataset = TensorDataset(features_tensor)

    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [20]:
def load_graph_data(filename):
    data = np.load(filename)
    adjacency_matrix = data['adjacency_matrix']
    keys = data['keys']
    values = data['values']
    index_to_iu_ac = {key: value for key, value in zip(keys, values)}
    return adjacency_matrix, index_to_iu_ac

def symmetric_normalize_adjacency(adjacency_matrix):
    D = np.diag(np.sqrt(np.sum(adjacency_matrix, axis=1)) + 1e-6)
    D_inv = np.linalg.inv(D)
    normalized_adjacency = np.dot(np.dot(D_inv, adjacency_matrix), D_inv)
    return normalized_adjacency


In [21]:
# train_loader = load_dataset(f'{inputdir}train_dataset_constructed.csv', feature_names)
train_loader = load_dataset(f'{inputdir}train_dataset_constructed_1500.csv', feature_names)
# eval_loader = load_dataset(f'{inputdir}eval_dataset_constructed.csv', feature_names)
# test_loader = load_dataset(f'{inputdir}test_dataset_constructed_x.csv', feature_names, train=False)

# 加载图数据
adjacency_matrix, index_to_iu_ac = load_graph_data(f"{inputdir}graph_data.npz")
# 归一化邻接矩阵
normalized_adjacency = symmetric_normalize_adjacency(adjacency_matrix)
adjacency_tensor = torch.from_numpy(normalized_adjacency)
adjacency_tensor = adjacency_tensor.to(device)


torch.Size([3503, 2, 4634, 12])


In [22]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
        return output + self.bias

class STGNN(nn.Module):
    def __init__(self, num_features, num_nodes, hidden_size, num_layers, dropout=0.5):
        super(STGNN, self).__init__()
        self.gc1 = GraphConvolution(num_features, hidden_size)
        # Now the input_size to LSTM should consider all nodes as part of the sequence
        self.lstm = nn.LSTM(input_size=num_nodes * hidden_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_nodes * num_timesteps * 1)  # 修改输出大小
        
    def forward(self, x, adj):
        # Apply the graph convolution to each time step
        batch_size, num_timesteps, num_nodes, num_features = x.shape
        x = x.view(batch_size * num_timesteps, num_nodes, num_features)
        x = F.relu(self.gc1(x, adj))
        x = x.view(batch_size, num_timesteps, -1)  # Flatten all node features into the feature dimension
        # Process sequences using LSTM
        x, _ = self.lstm(x)  # LSTM expects [batch_size, seq_length, input_size]
        # Take the output from the last time step and reshape it
        x = self.fc(x[:, -1, :])
        x = x.view(batch_size, num_timesteps, num_nodes, 1)  # Reshape to desired output shape
        return x

In [23]:
def train_model(model, train_loader, adjacency_matrix, criterion, optimizer, num_epochs, log_interval=10, checkpoint_dir='./logs/stgnn/',TBlog_dir='./runs/stgnn/'):
    model.to(device)
    
    # 初始化 TensorBoard 记录器
    if not os.path.exists(TBlog_dir):
        os.makedirs(TBlog_dir)
    writer = SummaryWriter(TBlog_dir)
    start_epoch = 1
    flag = True
    checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pth')  # 指定检查点文件名
    
    # 确保检查点目录存在
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    # 检查是否存在检查点
    if os.path.exists(checkpoint_path):
        print("Loading checkpoint...")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1  # 从下一个周期开始
        
    model.train()
    for epoch in range(start_epoch, num_epochs + start_epoch):
        # for batch_idx, (inputs, targets) in tqdm(enumerate(train_loader), desc="Training"):
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            if flag:
                print(next(model.parameters()).is_cuda)  # 确认模型参数是否在 GPU 上
                print(inputs.is_cuda)  # 确认输入是否在 GPU 上
                flag = False
            optimizer.zero_grad()
            output = model(inputs, adjacency_matrix)
            # print(f'output: {output.shape}, target:{targets.shape}')
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            # 在 TensorBoard 中记录损失
            if batch_idx % log_interval == 0:
                writer.add_scalar('Loss/train', loss.item(), epoch * len(train_loader) + batch_idx)
            
            # print(f'Epoch {epoch}, Batch {batch_idx+1}, Loss: {loss.item()}')
            # 定期保存检查点
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx+1}, Loss: {loss.item()}')
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss.item(),
                }, checkpoint_path)
    # 关闭 TensorBoard 记录器
    writer.close()


In [24]:
model = STGNN(len(feature_names), num_nodes=num_nodes, hidden_size=64, num_layers=2).to(device)
optimizer = Adam(model.parameters(), lr=0.01)
criterion = nn.L1Loss()

In [25]:
train_model(model, train_loader, adjacency_tensor, criterion, optimizer, num_epochs=10)

True
True
Epoch 1, Batch 1, Loss: 243.07518005371094
Epoch 2, Batch 1, Loss: 242.43093872070312
Epoch 3, Batch 1, Loss: 242.03390502929688
Epoch 4, Batch 1, Loss: 241.29771423339844
Epoch 5, Batch 1, Loss: 240.2039337158203
Epoch 6, Batch 1, Loss: 239.1455078125
Epoch 7, Batch 1, Loss: 238.21034240722656
Epoch 8, Batch 1, Loss: 237.3214874267578
Epoch 9, Batch 1, Loss: 236.4547576904297
Epoch 10, Batch 1, Loss: 235.60665893554688


In [27]:
eval_loader = load_dataset(f'{inputdir}eval_dataset_constructed.csv', feature_names)
# # test_loader = load_dataset(f'{inputdir}test_dataset_constructed_x.csv', feature_names, train=False)

ValueError: cannot reshape array of size 17071656 into shape (2,4634,12)

In [ ]:
# def calculate_mae(model, data_loader,adjacency_matrix= adjacency_matrix, device=device):
#     model.eval()  
#     total_mae = 0.0
#     total_count = 0
    
#     with torch.no_grad():  
#         for inputs, targets in data_loader:
#             inputs, targets = inputs.to(device), targets.to(device)
#             output = model(inputs, adjacency_matrix)
#             mae = l1_loss(output, targets, reduction='sum')
#             total_mae += mae.item()
#             total_count += targets.size(0)
    
#     average_mae = total_mae / total_count
#     return average_mae


# average_mae = calculate_mae(model, eval_loader)
# print(f'Average MAE on evaluation data: {average_mae:.4f}')

In [ ]:
# def evaluate_model(model, loader, adjacency_matrix= adjacency_matrix, device=device):
#     model.eval()
#     predictions = []
#     with torch.no_grad():
#         for features, _ in loader:
#             features = features[0].to(device)
#             output = model(features, adjacency_matrix)
#             # predictions.extend(outputs.cpu().numpy())
#             predictions.extend(output.round().cpu().numpy())
#     return predictions

# predictions = evaluate_model(model, eval_loader)

# predictions_df = pd.DataFrame(predictions)
# predictions_df.index = predictions_df.index + 1  # Adjust index to start from 1
# print(len(predictions_df))
# # Save the predictions to a CSV file
# predictions_df.to_csv(f'{resultdir}stgnn_predictions.csv', index_label='id')

# print("Predictions saved to 'stgnn_predictions.csv', with IDs starting from 1.")